## 문제정의
고객이 계정을 유지할지, 아니면 해지(이탈)할지를 예측

**컬럼해석** 
- CustomerID: 고객 고유 식별자
- Surname: 고객 성(Last name)
- CreditScore: 신용 점수(숫자)
- Geography: 거주 국가(France/Spain/Germany)
- Gender: 성별(Male/Female)
- Age: 나이
- Tenure: 은행 이용 기간(연수)
- Balance: 계좌 잔액
- NumOfProducts: 이용 중인 은행 상품 수(예: 예금, 카드 등)
- HasCrCard: 신용카드 보유 여부(1=예, 0=아니오)
- IsActiveMember: 활성 회원 여부(1=예, 0=아니오)
- EstimatedSalary: 추정 급여(연봉)
- Exited: 이탈 여부(1=이탈, 0=유지) (대회 타깃 변수)지) (대회 타깃 변수)

In [1]:
import pandas as pd
train = pd.read_csv("origin/train-checkpoint.csv")
test = pd.read_csv('origin/test-checkpoint.csv')
test
train

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


## 데이터 전처리

In [2]:
train.shape, test.shape # ((165034, 14), (110023, 13))

((165034, 14), (110023, 13))

### train data 확인

In [3]:
train.info() # Surname, Geography, Gender

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


In [4]:
train.describe() # 수치형 컬럼

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [5]:
train.describe(include="O") # 범주형 컬럼

,Surname,Geography,Gender
count,165034,165034,165034
unique,2797,3,2
top,Hsia,France,Male
freq,2456,94215,93150


In [6]:
train['Surname'].value_counts()

Surname
Hsia         2456
T'ien        2282
Hs?          1611
Kao          1577
Maclean      1577
             ... 
Samaniego       1
Lawley          1
Bonwick         1
Tennant         1
Elkins          1
Name: count, Length: 2797, dtype: int64

In [7]:
train['Geography'].value_counts() # France, Spain, Germany

Geography
France     94215
Spain      36213
Germany    34606
Name: count, dtype: int64

In [8]:
train['Gender'].value_counts() # Male, Female

Gender
Male      93150
Female    71884
Name: count, dtype: int64

In [9]:
train.isnull().sum() # 결측치 확인 -> 결측치 없음

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### test data 확인

In [10]:
test.info() # Surname, Geography, Gender -> object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110023 entries, 0 to 110022
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               110023 non-null  int64  
 1   CustomerId       110023 non-null  int64  
 2   Surname          110023 non-null  object 
 3   CreditScore      110023 non-null  int64  
 4   Geography        110023 non-null  object 
 5   Gender           110023 non-null  object 
 6   Age              110023 non-null  float64
 7   Tenure           110023 non-null  int64  
 8   Balance          110023 non-null  float64
 9   NumOfProducts    110023 non-null  int64  
 10  HasCrCard        110023 non-null  float64
 11  IsActiveMember   110023 non-null  float64
 12  EstimatedSalary  110023 non-null  float64
dtypes: float64(5), int64(5), object(3)
memory usage: 10.9+ MB


In [11]:
test.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,110023.000000,1.100230e+05,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000
mean,220045.000000,1.569210e+07,656.530789,38.122205,4.996637,55333.611354,1.553321,0.753043,0.495233,112315.147765
std,31761.048671,7.168499e+04,80.315415,8.861550,2.806148,62788.519675,0.544714,0.431244,0.499980,50277.048244
min,165034.000000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000
25%,192539.500000,1.563286e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74440.325000
50%,220045.000000,1.569018e+07,660.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117832.230000
75%,247550.500000,1.575693e+07,710.000000,42.000000,7.000000,120145.605000,2.000000,1.000000,1.000000,154631.350000
max,275056.000000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000


In [12]:
test.describe(include="O")

,Surname,Geography,Gender
count,110023,110023,110023
unique,2708,3,2
top,Hsia,France,Male
freq,1606,63171,61942


In [13]:
test['Surname'].value_counts()

Surname
Hsia         1606
T'ien        1484
Hs?          1124
Maclean      1042
Ts'ui        1017
             ... 
Wallwork        1
Praed           1
Kentish         1
Younger         1
Distefano       1
Name: count, Length: 2708, dtype: int64

In [14]:
test['Geography'].value_counts() # France, Spain, Germany

Geography
France     63171
Spain      23913
Germany    22939
Name: count, dtype: int64

In [15]:
test['Gender'].value_counts() # Male, Female

Gender
Male      61942
Female    48081
Name: count, dtype: int64

In [16]:
test.isnull().sum() # 결측치 없음

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

## 인코딩

In [17]:
# 아래 레이블 인코딩에서 test에서 Lucchese라는 문자열을 숫자로 변경할 수 X -> 값을 0으로 지정
# import numpy as np

# test[test['Surname'] == 'Lucchese']
# test['Surname'] = pd.to_numeric(test['Surname'], errors='coerce')
# test['Surname'] = test['Surname'].fillna(0)

In [18]:
y_train = train.pop('Exited') # y_train : 종속변수 생성

In [19]:
y_train.value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

In [20]:
# 레이블 인코딩 : Surname, Geography, Gender 범주형 변수 -> 수치화
from sklearn.preprocessing import LabelEncoder

cols = train.columns[train.dtypes == object]
# cols = ['Geography', 'Gender']
for col in cols:
    le = LabelEncoder() # 각 열마다 독립적인 인코딩 규칙을 적용
    le.fit(pd.concat([train[col], test[col]], axis=0))
    train[col] = le.transform(train[col]) # fit_transform() : 학습 및 변환
    test[col] = le.transform(test[col]) # transform() : 변환만 적용

# fit() : 학습데이터인 train[col] 의 고유값을 분석하여 매핑규칙(인코딩 규칙)을 학습
# -> transform() : 학습된 규칙에 따라 train[col]의 값을 숫자로 변환하고 그 결과를 다시 train[col]에 다시 저장

# test[col] 에는 학습된(fit()) 동일한 규칙을 적용하여 숫자로 반환
# 테스트 데이터에는 fit() 사용하지 X !!!!!

## Surname에 대한 문제가 자꾸 발생해서 Surname 제외한 Geography, Gender만 cols에 넣어줌

In [21]:
train.head()  # Surname, Geography, Gender 수치형으로 된 것 확인

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0,15674932,1992,668,0,1,33.0,3,0.00,2,1.0,0.0,181449.97
1,1,15749177,1993,627,0,1,33.0,1,0.00,2,1.0,1.0,49503.50
2,2,15694510,1217,678,0,1,40.0,10,0.00,2,1.0,0.0,184866.69
3,3,15741417,1341,581,0,1,34.0,2,148882.54,1,1.0,1.0,84560.88
4,4,15766172,483,716,2,1,33.0,5,0.00,2,1.0,1.0,15068.83


In [22]:
test.head() # Surname, Geography, Gender 수치형으로 된 것 확인

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,1581,586,0,0,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,1935,683,0,0,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,1331,656,0,0,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,1955,681,0,1,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,1153,752,1,1,38.0,10,121263.62,1,1.0,0.0,139431.00


In [23]:
test['Geography'].value_counts() # France(0), Spain(2), Germany(1)

Geography
0    63171
2    23913
1    22939
Name: count, dtype: int64

In [24]:
test['Gender'].value_counts() # Male(1), Female(0)

Gender
1    61942
0    48081
Name: count, dtype: int64

## 모델 학습 및 평가

In [25]:
# 검증데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((132027, 13), (33007, 13), (132027,), (33007,))

In [ ]:
# 모델 학습
# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred1 = rf.predict_proba(X_val) # 클래스별 확률 예측
pred1[:,1]

In [27]:
# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
pred2 = lgbmc.predict_proba(X_val)
pred2[:,1]

array([0.06442807, 0.14527317, 0.68723293, ..., 0.23911734, 0.02022277,
       0.30957594])

In [28]:
# 성능 평가 - 랜덤 포레스트
pred1_label = rf.predict(X_val) 
from sklearn.metrics import roc_auc_score
roc_auc1 = roc_auc_score(y_val, pred1[:,1])
print('roc_auc:', roc_auc1)

from sklearn.metrics import accuracy_score
accuracy1 = accuracy_score(y_val, pred1_label)
print('accuracy_score:', accuracy1)

from sklearn.metrics import f1_score
f1_1 = f1_score(y_val, pred1_label, pos_label=1) # 이탈한 고객 : 1 로 처리했기 때문에 pos_label 바꿔주기
print('f1_score:', f1_1)

roc_auc: 0.8801463308572731
accuracy_score: 0.8627260884054897
f1_score: 0.6168935486598461


In [29]:
# 성능 평가 - LightGBM
pred2_label = rf.predict(X_val)
from sklearn.metrics import roc_auc_score
roc_auc2 = roc_auc_score(y_val, pred2[:,1])
print('roc_auc:', roc_auc2)

from sklearn.metrics import accuracy_score
accuracy2 = accuracy_score(y_val, pred2_label)
print('accuracy_score:', accuracy2)

from sklearn.metrics import f1_score
f1_2 = f1_score(y_val, pred2_label, pos_label=1) # 이탈한 고객 : 1 로 처리했기 때문에 pos_label 바꿔주기
print('f1_score:', f1_2)

roc_auc: 0.8902162094130004
accuracy_score: 0.8627260884054897
f1_score: 0.6168935486598461


In [30]:
# 파일로 저장
pred = lgbmc.predict_proba(test)
lgbmc.classes_
submit = pd.DataFrame({'pred' : pred[:, 1]})
submit.to_csv('ex_result.csv', index=False)
pd.read_csv('ex_result.csv')

,pred
0,0.024092
1,0.814112
2,0.027926
3,0.220899
4,0.341188
...,...
110018,0.036409
110019,0.165881
110020,0.017981
110021,0.171473
